                                            Assignment 2 
                                      task 1 (Data Collection)

                                      Team Members - 
                                      1. Ganesh Nimkar (K00277641)
                                      2. Sanket Sangar (K00277641)

First import all required libraries

In [1]:
import sqlite3
from sqlite3 import Error
import requests
import pandas as pd
import traceback
import time
import json


2.6.0


Create a .db file in local and give path to connect to that .db

In [ ]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
    finally:
        if conn:
            conn.close()


if __name__ == '__main__':
    create_connection(r"../bike.db")

Now we create a endpoint string to be called to get the data 

In [2]:
contract_base_url = 'https://api.jcdecaux.com/vls/v3/contracts?'  #contracts data
key = '5bab97f2efa030252fda733dfa249f9357866aa5'   #Unique key for every user
stations_base_url = 'https://api.jcdecaux.com/vls/v3/stations?'

contract_url = contract_base_url + 'apiKey=' + key

First, we call /contracts API to get the contracts data. This includes name of contract (city), country code etc fields. We are interested in name and country_code field.  

In [3]:
contracts_response = response = requests.get(contract_url) 
contracts_response.json()

[{'name': 'rouen',
  'commercial_name': "cy'clic",
  'cities': ['Rouen'],
  'country_code': 'FR'},
 {'name': 'jcdecauxbike',
  'commercial_name': None,
  'cities': None,
  'country_code': None},
 {'name': 'toulouse',
  'commercial_name': 'Vélô',
  'cities': ['Toulouse'],
  'country_code': 'FR'},
 {'name': 'luxembourg',
  'commercial_name': 'Veloh',
  'cities': ['Luxembourg'],
  'country_code': 'LU'},
 {'name': 'dublin',
  'commercial_name': 'dublinbikes',
  'cities': ['Dublin'],
  'country_code': 'IE'},
 {'name': 'valence',
  'commercial_name': None,
  'cities': None,
  'country_code': None},
 {'name': 'stockholm',
  'commercial_name': 'Cyclocity',
  'cities': ['Stockholm'],
  'country_code': 'SE'},
 {'name': 'santander',
  'commercial_name': 'Tusbic',
  'cities': ['Santander'],
  'country_code': 'ES'},
 {'name': 'lund',
  'commercial_name': 'Lundahoj',
  'cities': ['Lund'],
  'country_code': 'SE'},
 {'name': 'maribor',
  'commercial_name': None,
  'cities': None,
  'country_code': Non

After we get the contracts data, we can loop through that to call the /stations data by passing contract name. We have created a table called "bikesData" and declared all the columns that this table will hold. 

Below code does the following actions step by step.

    1. Loop throught the contracts data. 
    2. pass the "name" attribute value to /stations API as contract. 
    3. Get the stations data and append to all_stations_data list. 
    4. after above step finishes, convert the list to dataframe using Panda's and pass it to add_to_database function. 
    5. This function takes the dataframe, iterates throught it. While iterating, we get the data from each column and create a list. We then insert this list into the database. 
    
### Above function runs for every 15 min. 

In [ ]:
all_stations_data = [] 
conn = sqlite3.connect("../bike.db") # Connect to database (creates if it does not exist)
cursor = conn.cursor()
cursor.execute("CREATE TABLE IF NOT EXISTS bikesdata (number integer, contractName text,name text,address text, position text, banking integer, bonus integer, status text, lastUpdate timestamp, connected integer, overflow integer, totalStands text, mainStands text, overflowStands text, country_code text)")
conn.commit() # Save the changes
print(cursor)
def add_to_database(dataframe):
    """ Function to add information to the database """
    
    # df.shape returns the number of columns and rows in a dataframe
    # So using the first value returned, we can cycle through each row in the dataframe (where each row has information on a specific station)
    for i in range(0, (dataframe.shape[0]-1)):
        data = dataframe.iloc[i] # df.iloc[] just allows us to access elements via normal indexing of a pandas dataframe
        
        # Store all the information from the dataframe in a list
        elements = [int(data.get("number")), data.get("contractName"), data.get("name"), data.get("address"), data.get("position"), int(data.get("banking")),  int(data.get("bonus")), data.get("status"), data.get("lastUpdate"), int(data.get("connected")), int(data.get("overflow")), data.get("totalStands"), data.get("mainStands"),data.get("overflowStands"), data.get("country_code")]
        
        # Add each of these elements to the table in our database
        cursor.execute("INSERT INTO bikesdata VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)", elements)
    conn.commit()
    
while True:
    localtime = time.localtime()
    result = time.strftime("%I-%M-%S%p", localtime)
    
    try:
        # Access info on JCDecaux website using API key
        # Convert the JSON information into a pandas dataframe format
        
        for i, contract in enumerate(contracts_response.json()): 
            station_url = stations_base_url + "contract="+contract["name"] + '&apiKey='+ key
            res = requests.get(station_url)
            stations_data = res.json()
            if(len(stations_data) > 0 ):
                for index, stData in enumerate(stations_data):
                    stations_data[index]["country_code"] = contract["country_code"]
                    stations_data[index]["position"] = json.dumps(stations_data[index]["position"])
                    stations_data[index]["totalStands"] = json.dumps(stations_data[index]["totalStands"])
                    stations_data[index]["mainStands"] = json.dumps(stations_data[index]["mainStands"])
                    stations_data[index]["overflowStands"] = json.dumps(stations_data[index]["overflowStands"])
                    all_stations_data.append(stations_data[index])
    
        df = pd.DataFrame(all_stations_data)
        # Add the information to the database
        add_to_database(df)
        all_stations_data = []
        print('time ', result)
        # Sleep for 5 minutes
        time.sleep(900)

    except:
        # Print traceback if there is an error
        print(traceback.format_exc())
        

time  01-43-41AM
time  01-58-47AM
time  02-13-52AM
time  02-28-58AM
time  02-44-04AM
time  03-00-37AM
time  03-15-44AM
time  03-30-51AM
time  03-45-56AM
time  04-01-03AM
time  04-16-11AM
time  04-31-17AM
time  04-46-23AM
time  05-01-29AM
time  05-16-59AM
time  05-32-05AM
time  05-47-12AM
time  06-02-20AM
time  06-17-27AM
time  06-32-34AM
time  06-47-40AM
time  07-02-46AM
time  07-17-56AM
time  07-33-02AM
time  07-48-10AM
time  08-03-16AM
time  08-18-24AM
time  08-33-29AM
time  08-48-38AM
time  09-04-09AM
time  09-19-15AM
time  09-34-21AM
time  09-49-28AM
time  10-04-35AM
time  10-19-42AM
time  10-34-48AM
time  10-49-53AM
time  11-04-58AM
time  11-20-04AM
time  11-35-11AM
time  11-50-22AM
time  12-05-28PM
time  12-20-35PM
time  12-35-41PM
time  12-50-55PM
time  01-06-01PM
time  01-21-07PM
time  01-36-14PM
time  01-51-21PM
time  02-06-27PM
time  02-21-33PM
time  02-36-39PM
time  02-51-45PM
time  03-06-51PM
time  03-21-56PM
time  03-37-01PM
time  03-52-07PM
time  04-07-14PM
time  04-22-20

We print the timestamp at which the API is called. 

## By this way we are storing data of all stations of all countries for every 15 min. 